In [13]:

import re
import numpy as np
import pandas as pd 
from pathlib import Path
# pykilosort 
from readSGLX import readMeta
from pykilosort import run, add_default_handler, np2_4shank_probe 


# get queue
root = r'\\zserver.cortexlab.net\Code\AVrig'
queue_csv_file = '%s\pykilosort_queue.csv' % root
queue_csv = pd.read_csv(queue_csv_file)
channel_map = np2_4shank_probe()
for _,rec in queue_csv.iterrows():
    #check if recording is not being sorted already 
    if rec.sortedTag==0: 

        try:
            input_dir = Path(rec.ephysName)
            
            # read the metadata and get the x- and ypos of channels 
            channel_data = re.findall(r"\d+:\d+:\d+:\d+",meta['snsShankMap'])
            shank_idx = np.array([np.array(re.split(':',chandat)).astype('int')[0] for chandat in channel_data])
            side_idx = np.array([np.array(re.split(':',chandat)).astype('int')[1] for chandat in channel_data])
            channel_map.xc = ((shank_idx*250) + side_idx*32).astype('float64')
            channel_map.yc = (np.array([np.array(re.split(':',chandat)).astype('int')[2] for chandat in channel_data])*15).astype('float64')
            pass 

            output_dir = input_dir.parents[0] / 'pyKS'
            output_dir.mkdir(parents=False, exist_ok=True)
            
            add_default_handler(level='INFO') # print output as the algorithm runs
            # do something with the sortedTag
            run(input_dir, probe=channel_map, low_memory=True, dir_path=output_dir)
        except: